In [1]:
import requests
from bs4 import BeautifulSoup
import os
import time
import random

### 得到每一页的全部项目链接（每10页操作一下）

In [3]:
# 定义目标URL（"办公建筑"分类页面）
url_root = "https://www.archdaily.com"

proj_links_all = []
proj_titles_all = []

count = 0
for i in range(0,10):
    # 每一页
    url = "https://www.archdaily.com/category/office-buildings/page/"+str(i+1)
    print(url)
    response = requests.get(url)
    content = response.content
    soup = BeautifulSoup(content, "html.parser")
    
    # 每一页各个项目链接
    proj_links = []
    proj_titles = []
    for li in soup.find_all('li',class_="afd-search-list__4xitem"):
        a = li.find('a', class_="afd-search-list__link")
        for a in li.find_all('a', class_="afd-search-list__link"):
            input_string = a['href']
            proj_title = a['href'].split("/")[2]
            url_proj = url_root + a['href']
            proj_links.append(url_proj)
            proj_titles.append(proj_title)
            count = count+1
    proj_links_all.append(proj_links)
    proj_titles_all.append(proj_titles)
    print("page "+str(i+1)+"   total projects num: "+str(count))
    time.sleep(2)


https://www.archdaily.com/category/office-buildings/page/1
page 1   total projects num: 24
https://www.archdaily.com/category/office-buildings/page/2
page 2   total projects num: 48
https://www.archdaily.com/category/office-buildings/page/3
page 3   total projects num: 72
https://www.archdaily.com/category/office-buildings/page/4
page 4   total projects num: 96
https://www.archdaily.com/category/office-buildings/page/5
page 5   total projects num: 120
https://www.archdaily.com/category/office-buildings/page/6
page 6   total projects num: 144
https://www.archdaily.com/category/office-buildings/page/7
page 7   total projects num: 168
https://www.archdaily.com/category/office-buildings/page/8
page 8   total projects num: 192
https://www.archdaily.com/category/office-buildings/page/9
page 9   total projects num: 216
https://www.archdaily.com/category/office-buildings/page/10
page 10   total projects num: 240


### 依次访问各个项目的链接

In [4]:
import re

In [ ]:
for i in range(6, 7):
    for j in range(14, len(proj_links_all[i])):
        try:
            title = proj_titles_all[i][j]
            link = proj_links_all[i][j]

            os.makedirs("ArchDailyImages/"+title, exist_ok=True)
            project_response = requests.get(link)
            project_content = project_response.content
            project_soup = BeautifulSoup(project_content, "html.parser")

            # 项目信息
            specs_li = project_soup.find_all("li", class_='afd-specs__item')
            specs_data = {}
            for li in specs_li:
                span_key =  li.find("span", class_='afd-specs__key')
                span_value =  li.find("span", class_='afd-specs__value')
                if span_key is not None:
                    key = re.sub(r'[\s\n]', '', span_key.get_text())
                    if span_value is not None:
                        value = re.sub(r'[\s\n]', '', span_value.get_text())
                        specs_data[key] = value
                    else:
                        specs_data[key] = ''
            # 保存为JSON文件
            specs_path = "ArchDailyImages/"+title+"/specs.json"
            with open(specs_path, "w") as json_file:
                json.dump(specs_data, json_file)

            # 项目图片
            proj_pic = project_soup.find_all('a', class_='gallery-thumbs-link')
            img_tags = []
            for k in range(0,len(proj_pic)):
                p = proj_pic[k]
                img =  p.find('img')
                img_tag = img['alt'].split("- ")[1]
                tag = img_tag.split(", ")
                img_tags.append(tag)

                img_link = img['src'].replace('medium_jpg', 'slideshow')
                image_response = requests.get(img_link)
                image_data = image_response.content
                with open(os.path.join("ArchDailyImages",title, f"{k+1}.jpg"), "wb") as f:
                    f.write(image_data)
                time.sleep(0.8) #存图间隔

            tags_path = "ArchDailyImages/"+title+"/img_tags.txt"
            # 将列表整理成指定格式的字符串
            formatted_data = ""
            for sublist in img_tags:
                formatted_data += ",".join(sublist) + "\n"

            # 将整理后的数据写入到本地文件
            with open(tags_path, "w") as file:
                file.write(formatted_data)

            time.sleep(1) #打开项目间隔
            print("page "+str(i+1)+" -- "+title+" saved. | specs: "+str(len(specs_data))+" | imgs: "+str(len(proj_pic)))
        except requests.exceptions.RequestException as e:
            print("Error processing at page: "+str(i+1)+" project: "+str(j+1))
            continue

page 7 -- awesome185-office-building-architects-group-raum saved. | specs: 0 | imgs: 0
page 7 -- ribeirao-preto-shared-fuel-base-office-estudio-cidade saved. | specs: 21 | imgs: 21


### 对每一个项目链接再次发送请求，解析项目信息和图片

#### 项目信息

In [84]:
project_link = "https://www.archdaily.com/1005735/the-white-renovation-office-building-olgooco"
project_response = requests.get(project_link)
project_content = project_response.content
project_soup = BeautifulSoup(project_content, "html.parser")

# 项目信息
specs_li = project_soup.find_all("li", class_='afd-specs__item')
data = {}
for li in specs_li:
    span_key =  li.find("span", class_='afd-specs__key')
    span_value =  li.find("span", class_='afd-specs__value')
    if span_key is not None:
        key = re.sub(r'[\s\n]', '', span_key.get_text())
        if span_value is not None:
            value = re.sub(r'[\s\n]', '', span_value.get_text())
            data[key] = value
        else:
            data[key] = ''
print(data)

{'Architects:': 'Olgooco', 'Area:': '3000m²', 'Year:': '2022', 'Photographs:': 'MohammadHassanEttefagh,ParhamTaghioff', 'Manufacturers:IvanSystem,SalcoCompany': '', 'LeadArchitects:': 'MehranKhoshroo', 'Contractor:': 'CandoEngineeringGroup', 'DesignTeam:': 'ReyhaneMiraftab-SaminMostafaei-SoroushZeydanian', 'Structure:': 'AfshinMasoudi', 'Electrical:': 'Mr.Mardani', 'Mechanical:': 'ImanEILBeigi', 'Graphic:': 'MahsaEsmaeili', 'Program/Use/BuildingFunction:': 'Officebuilding', 'Client:': 'PayaWealthManagement', 'Supervision:': 'OlgooOffice', 'City:': 'Tehran', 'Country:': 'Iran'}


#### 项目图片

In [112]:
proj_pic = project_soup.find_all('a', class_='gallery-thumbs-link')
print(len(proj_pic))

img_tags = []
for i in range(0,len(proj_pic)):
    p = proj_pic[i]
    img =  p.find('img')
    img_tag = img['alt'].split("- ")[1]
    tag = img_tag.split(", ")
    img_tags.append(tag)
    
    img_link = img['src'].replace('medium_jpg', 'slideshow')
    
    print(img_link)
    
    image_response = requests.get(img_link)
    image_data = image_response.content
    with open(os.path.join("ArchDailyImages/the-white-renovation-office-building-olgooco", f"{i+1}.jpg"), "wb") as f:
        f.write(image_data)
        
    time.sleep(1)

print(img_tags)


filename = "img_tags.txt"
# 将列表整理成指定格式的字符串
formatted_data = ""
for sublist in img_tags:
    formatted_data += ",".join(sublist) + "\n"

# 将整理后的数据写入到本地文件
with open(filename, "w") as file:
    file.write(formatted_data)

36
https://images.adsttc.com/media/images/64e3/a79c/fbc6/582d/9274/f217/slideshow/the-white-renovation-office-building-olgooco_1.jpg?1692641200
https://images.adsttc.com/media/images/64e3/a7a3/fbc6/582d/9274/f21b/slideshow/the-white-renovation-office-building-olgooco_13.jpg?1692641219
https://images.adsttc.com/media/images/64e3/a7b1/8177/ff72/a9c8/44f0/slideshow/the-white-renovation-office-building-olgooco_18.jpg?1692641216
https://images.adsttc.com/media/images/64e3/a7a2/fbc6/582d/9274/f219/slideshow/the-white-renovation-office-building-olgooco_17.jpg?1692641203
https://images.adsttc.com/media/images/64e3/a7a3/fbc6/582d/9274/f21a/slideshow/the-white-renovation-office-building-olgooco_2.jpg?1692641221
https://images.adsttc.com/media/images/64e3/a79f/8177/ff72/a9c8/44ec/slideshow/the-white-renovation-office-building-olgooco_16.jpg?1692641208
https://images.adsttc.com/media/images/64e3/a57e/8177/ff72/a9c8/44da/slideshow/the-white-renovation-office-building-olgooco_1.jpg?1692640645
https:

In [76]:
# 项目介绍文本
paras = project_soup.find_all("p",{'class_': ''})
print(paras)

[<p class="afd-mobile-navigation__item-title">
Projects
</p>, <p class="afd-mobile-navigation__item-title">
Images
</p>, <p class="afd-mobile-navigation__item-title">
Products
</p>, <p class="afd-mobile-navigation__item-title">
Folders
</p>, <p class="afd-mobile-navigation__item-title avatar-title">
Feed
</p>, <p class="js-tooltip-text"></p>, <p class="thumbs afd-desktop-e clearfix"><a class="thumbs__link" data-nr-picture-id="64e3a7a3fbc6582d9274f21b" href="/1005735/the-white-renovation-office-building-olgooco/64e3a7a3fbc6582d9274f21b-the-white-renovation-office-building-olgooco-photo" rel="attachment"><img alt="The White Renovation Office Building / Olgooco - Interior Photography, Chair" class="thumbs__img b-lazy" data-nr-picture-id="64e3a7a3fbc6582d9274f21b" data-pin-nopin="true" data-src="https://images.adsttc.com/media/images/64e3/a7a3/fbc6/582d/9274/f21b/thumb_jpg/the-white-renovation-office-building-olgooco_13.jpg?1692641219" height="125" longdesc="https://www.archdaily.com/10057